In [1]:
import pandas as pd

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession 
from pyspark.ml.feature import StringIndexer,VectorAssembler

spark = SparkSession.builder\
            .appName("Spark Dataframe Intro")\
            .getOrCreate()


Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [2]:
auto= spark.read.option("inferSchema","true").option("header","true").csv("/content/auto-mpg.data.txt")

auto.show(4)

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|       130|  3504|        12.0|        70|     1|chevrolet chevell...|
|15.0|        8|       350.0|       165|  3693|        11.5|        70|     1|   buick skylark 320|
|18.0|        8|       318.0|       150|  3436|        11.0|        70|     1|  plymouth satellite|
|16.0|        8|       304.0|       150|  3433|        12.0|        70|     1|       amc rebel sst|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
only showing top 4 rows



In [3]:
auto.dtypes

# casts all columns to a numeric type
from pyspark.sql.functions import col  
# for indicating a column using a string in the line below

auto = auto.select([col(c).cast("double").alias(c) for c in auto.columns])

auto.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: double (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- model_year: double (nullable = true)
 |-- origin: double (nullable = true)
 |-- name: double (nullable = true)



In [4]:
columns=["mpg","cylinders","displacement","horsepower","weight","acceleration","model_year","origin","name"]



auto.filter("mpg is null").show()
auto.filter("cylinders is null").show()
auto.filter("displacement is null").show()
auto.filter("horsepower is null").show()
auto.filter("weight is null").show()
auto.filter("acceleration is null").show()
auto.filter("model_year is null").show()
auto.filter("origin is null").show()



+---+---------+------------+----------+------+------------+----------+------+----+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|
+---+---------+------------+----------+------+------------+----------+------+----+
+---+---------+------------+----------+------+------------+----------+------+----+

+---+---------+------------+----------+------+------------+----------+------+----+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|
+---+---------+------------+----------+------+------------+----------+------+----+
+---+---------+------------+----------+------+------------+----------+------+----+

+---+---------+------------+----------+------+------------+----------+------+----+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|
+---+---------+------------+----------+------+------------+----------+------+----+
+---+---------+------------+----------+------+------------+----------+------+----+

+

In [7]:
from pyspark.ml.feature import Imputer
medianImputer = Imputer(inputCol="horsepower",outputCol="horsepower_imputed_median",strategy="median")
auto = medianImputer.fit(auto).transform(auto)
auto.show(2)

+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|horsepower_imputed_median|
+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|      70.0|   1.0|null|                    130.0|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|      70.0|   1.0|null|                    165.0|
+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+
only showing top 2 rows



In [8]:
auto.filter("mpg is null").show()
auto.filter("cylinders is null").show()
auto.filter("displacement is null").show()
auto.filter("horsepower_imputed_median is null").show()
auto.filter("weight is null").show()
auto.filter("acceleration is null").show()
auto.filter("model_year is null").show()
auto.filter("origin is null").show()

+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|horsepower_imputed_median|
+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+
+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+

+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+
|mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|horsepower_imputed_median|
+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+
+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+

+---+---------+------------+----------+------+------------+----------+------+----+-------------------------+
|mpg|cylinders|di

In [9]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer,VectorAssembler,OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier


In [10]:
#indxr = StringIndexer(inputCol="City",outputCol="city_indexed")
#placesDF = indxr.fit(placesDF).transform(placesDF)

encoder = OneHotEncoder(inputCol="origin",outputCol="origin_class")
auto = encoder.fit(auto).transform(auto)

auto.show(2)


+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+-------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|name|horsepower_imputed_median| origin_class|
+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+-------------+
|18.0|      8.0|       307.0|     130.0|3504.0|        12.0|      70.0|   1.0|null|                    130.0|(3,[1],[1.0])|
|15.0|      8.0|       350.0|     165.0|3693.0|        11.5|      70.0|   1.0|null|                    165.0|(3,[1],[1.0])|
+----+---------+------------+----------+------+------------+----------+------+----+-------------------------+-------------+
only showing top 2 rows



Origin class bir categoric deger bunu encoding ettim ve name işime yarayacak bir kolon olmadıgı ıcın onu artık almadım

In [11]:
auto=auto.select("mpg","cylinders","displacement","horsepower_imputed_median", "weight", "acceleration", "model_year","origin_class")
auto.show(4)

+----+---------+------------+-------------------------+------+------------+----------+-------------+
| mpg|cylinders|displacement|horsepower_imputed_median|weight|acceleration|model_year| origin_class|
+----+---------+------------+-------------------------+------+------------+----------+-------------+
|18.0|      8.0|       307.0|                    130.0|3504.0|        12.0|      70.0|(3,[1],[1.0])|
|15.0|      8.0|       350.0|                    165.0|3693.0|        11.5|      70.0|(3,[1],[1.0])|
|18.0|      8.0|       318.0|                    150.0|3436.0|        11.0|      70.0|(3,[1],[1.0])|
|16.0|      8.0|       304.0|                    150.0|3433.0|        12.0|      70.0|(3,[1],[1.0])|
+----+---------+------------+-------------------------+------+------------+----------+-------------+
only showing top 4 rows



Mpg targetım oldugu ıcın adını label olarak degıstırdım .

In [12]:
auto = auto.withColumnRenamed("mpg","label")
auto.show(2)

+-----+---------+------------+-------------------------+------+------------+----------+-------------+
|label|cylinders|displacement|horsepower_imputed_median|weight|acceleration|model_year| origin_class|
+-----+---------+------------+-------------------------+------+------------+----------+-------------+
| 18.0|      8.0|       307.0|                    130.0|3504.0|        12.0|      70.0|(3,[1],[1.0])|
| 15.0|      8.0|       350.0|                    165.0|3693.0|        11.5|      70.0|(3,[1],[1.0])|
+-----+---------+------------+-------------------------+------+------------+----------+-------------+
only showing top 2 rows



In [13]:
vec = VectorAssembler(inputCols=auto.columns[1:10],outputCol="features")
auto =vec.transform(auto)
auto=auto.select("features","label")
auto.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[8.0,307.0,130.0,...| 18.0|
|[8.0,350.0,165.0,...| 15.0|
|[8.0,318.0,150.0,...| 18.0|
|[8.0,304.0,150.0,...| 16.0|
|[8.0,302.0,140.0,...| 17.0|
|[8.0,429.0,198.0,...| 15.0|
|[8.0,454.0,220.0,...| 14.0|
|[8.0,440.0,215.0,...| 14.0|
|[8.0,455.0,225.0,...| 14.0|
|[8.0,390.0,190.0,...| 15.0|
|[8.0,383.0,170.0,...| 15.0|
|[8.0,340.0,160.0,...| 14.0|
|[8.0,400.0,150.0,...| 15.0|
|[8.0,455.0,225.0,...| 14.0|
|[4.0,113.0,95.0,2...| 24.0|
|[6.0,198.0,95.0,2...| 22.0|
|[6.0,199.0,97.0,2...| 18.0|
|[6.0,200.0,85.0,2...| 21.0|
|[4.0,97.0,88.0,21...| 27.0|
|[4.0,97.0,46.0,18...| 26.0|
+--------------------+-----+
only showing top 20 rows



In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import Imputer


lin_Reg_ = LinearRegression(maxIter=10)

params = ParamGridBuilder() \
    .addGrid(lin_Reg_.regParam, [0, 0.01, 0.05, 0.1, 0.5, 1]) \
    .build()
eva = MulticlassClassificationEvaluator(metricName="accuracy")

validator = CrossValidator(
                                estimatorParamMaps=params,
                                estimator=lin_Reg_,
                                evaluator=eva,
                                numFolds=5,
                                parallelism=2
                          )



trainDF,testDF = auto.randomSplit([0.75,0.25],seed=1234)


model = validator.fit(trainDF)
resultDF = model.transform(testDF)

accuracy = eva.evaluate(resultDF)
print(accuracy)

0.0


In [40]:
from pyspark.ml.regression import LinearRegression 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

lrCV = LinearRegression(featuresCol='features', labelCol='label', maxIter=50)

#Creating a grid of parameter values that the cross-validation will use

paramGrid = ParamGridBuilder() \
  .addGrid(lrCV.regParam, [1, 0.1, 0.01]) \
  .addGrid(lrCV.fitIntercept, [True, False]) \
  .build()

#Create an instance of the CrossValidator object and enter our predefined parameters

crossVal = CrossValidator(estimator=lrCV,
                         estimatorParamMaps=paramGrid,
                         evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                         numFolds=5)

eva=MulticlassClassificationEvaluator(metricName="accuracy")
model = validator.fit(trainDF)
resultDF = model.transform(testDF)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(resultDF))

R Squared (R2) on test data = 0.775962
